In [23]:
import numpy as np
import numpy.linalg as lg
import matplotlib.pyplot as plt
from abc import ABCMeta, abstractmethod
from Matern import Matern
from Adaptor import Adaptor_xy
import math
import os
import copy
import json

In [24]:
class Archive(metaclass=ABCMeta):
    
    @abstractmethod
    def load_priors(self, paths: dict):
        pass


    @abstractmethod
    def read_experiences(self, exps):
        pass


    @abstractmethod
    def gibbs_sweep(self):
        pass


    @abstractmethod
    def MAP(self):
        pass


    @abstractmethod
    def save(self, dir_path=None):
        pass


    @abstractmethod
    def load(self, dir_path):
        pass




In [25]:
class Archive_xy(Archive):
    def __init__(self, alpha=1.0, threshold=40):
        self.alpha = alpha
        self.kernel = Matern(v=1.5)
        self.exps = {}
        self.next_exp_id = 1
        self.next_cluster_id = 1
        self.empty_cluster_ids = []
        self.empty_exp_ids = []
        self.clusters = {}
        self.threshold = threshold
        self.backup_clusters = {}
        self.x_priors = None
        self.y_priors = None
        self.log_prob = -np.inf

    
    def load_priors(self, paths):
        x_priors = np.load(paths["x"])
        self.x_prior_data = x_priors.copy()
        y_priors = np.load(paths["y"])
        self.y_prior_data = y_priors.copy()

        self.x_priors = []
        for x_prior in x_priors:
            coef = x_prior[:5].reshape(-1, 1)
            length_scale = x_prior[5:10]
            prior_var, noise_ratio = x_prior[10:12]
            noisy = True if x_prior[-1] else False
            self.x_priors.append((noisy, coef, length_scale, prior_var, noise_ratio))
        
        self.y_priors = []
        for y_prior in y_priors:
            coef = y_prior[:5].reshape(-1, 1)
            length_scale = y_prior[5:10]
            prior_var, noise_ratio = y_prior[10:12]
            noisy = True if y_prior[-1] else False
            self.y_priors.append((noisy, coef, length_scale, prior_var, noise_ratio))
    

    def read_experiences(self, exps, ids=None):
        if ids is None and exps:
            self.log_prob = -np.inf

        for id_index, exp in enumerate(exps):
            exp_dict = {}
            x_data = exp["x"]
            y_data = exp["y"]
            exp_dict["data_size"] = len(x_data)
            exp_dict["x_data"] = x_data.copy()
            exp_dict["y_data"] = y_data.copy()

            if not ids is None:
                exp_id = ids[id_index]
            elif self.empty_exp_ids:
                exp_id = self.empty_exp_ids[0]
                del self.empty_exp_ids[0]
            else:
                exp_id = self.next_exp_id
                self.next_exp_id += 1

            exp_dict["id"] = exp_id
            self.exps[exp_id] = exp_dict
            x_loglikelihoods = np.zeros(len(self.x_priors), dtype=np.float32)
            y_loglikelihoods = np.zeros(len(self.y_priors), dtype=np.float32)

            x_coors = x_data[:, :-1]
            x_target = x_data[:, -1].reshape(-1, 1)
            y_coors = y_data[:, :-1]
            y_target = y_data[:, -1].reshape(-1, 1)
            
            for index, (noisy, coef, length_scale, prior_var, noise_ratio) in enumerate(self.x_priors):
                diff = x_target - x_coors @ coef
                data_size = len(x_data)
                if noisy:
                    x_loglikelihood = -0.5*(diff / (prior_var*noise_ratio + prior_var)).T @ diff - \
                        0.5*data_size*math.log(prior_var*noise_ratio + prior_var)
                else:
                    cov = self.kernel(x_coors, length_scales=length_scale) + np.eye(data_size)*noise_ratio
                    cov *= prior_var
                    x_loglikelihood = -0.5*(diff.T @ lg.inv(cov) @ diff)
                    sign, log_det = lg.slogdet(cov)
                    log_det = log_det if sign else -100
                    x_loglikelihood -= 0.5*log_det
                x_loglikelihoods[index] = float(x_loglikelihood)

            for index, (noisy, coef, length_scale, prior_var, noise_ratio) in enumerate(self.y_priors):
                diff = y_target - y_coors @ coef
                data_size = len(y_data)
                if noisy:
                    y_loglikelihood = -0.5*(diff / (prior_var*noise_ratio + prior_var)).T @ diff - \
                        0.5*data_size*math.log(prior_var*noise_ratio + prior_var)
                else:
                    cov = self.kernel(y_coors, length_scales=length_scale) + np.eye(data_size)*noise_ratio
                    cov *= prior_var
                    y_loglikelihood = -0.5*(diff.T @ lg.inv(cov) @ diff)
                    sign, log_det = lg.slogdet(cov)
                    log_det = log_det if sign else -100
                    y_loglikelihood -= 0.5*log_det
                y_loglikelihoods[index] = float(y_loglikelihood)
            exp_dict["x_loglikelihoods"] = x_loglikelihoods
            exp_dict["y_loglikelihoods"] = y_loglikelihoods
            log_self_prob = np.log(np.mean(
                np.exp(x_loglikelihoods - np.max(x_loglikelihoods))
                )*np.mean(np.exp(y_loglikelihoods - np.max(y_loglikelihoods))))
            log_self_prob += np.max(x_loglikelihoods) + np.max(y_loglikelihoods)
            exp_dict["log_self_prob"] = log_self_prob

            if ids is None:
                exp_dict["cluster"] = self.alloc_cluster(exp_id)
            else:
                exp_dict["cluster"] = 0
        
    

    def drop_exp(self, exp_id):
        exp = self.exps[exp_id]
        self.remove_exp_from_cluster(exp_id, exp["cluster"])
        del self.exps[exp_id]
        self.empty_exp_ids.append(exp_id)
        self.log_prob = -np.inf



    def alloc_cluster(self, exp_id):
        if self.empty_cluster_ids:
            cluster_id = self.empty_cluster_ids[0]
            del self.empty_cluster_ids[0]
        else:
            cluster_id = self.next_cluster_id
            self.next_cluster_id += 1
        
        cluster_dict = {"id": cluster_id}
        cluster_dict["members"] = {exp_id}
        x_loglikelihoods = self.exps[exp_id]["x_loglikelihoods"].copy()
        y_loglikelihoods = self.exps[exp_id]["y_loglikelihoods"].copy()
        cluster_dict["x_loglikelihoods"] = x_loglikelihoods
        cluster_dict["y_loglikelihoods"] = y_loglikelihoods
        x_weights = np.exp(x_loglikelihoods - np.max(x_loglikelihoods))
        y_weights = np.exp(y_loglikelihoods - np.max(y_loglikelihoods))
        cluster_dict["x_weights"] = x_weights / np.sum(x_weights)
        cluster_dict["y_weights"] = y_weights / np.sum(y_weights)
        cluster_dict["size"] = 1
        cluster_dict["data_size"] = self.exps[exp_id]["data_size"]
        self.clusters[cluster_id] = cluster_dict
        if cluster_dict["data_size"] >= self.threshold:
            cluster_dict["large"] = True
            cluster_dict["x_kernel_index"] = int(np.argmax(x_weights))
            cluster_dict["y_kernel_index"] = int(np.argmax(y_weights))
            x_coef, y_coef = self.fit_coef(cluster_id)
            cluster_dict["x_coef"] = x_coef
            cluster_dict["y_coef"] = y_coef
        else:
            cluster_dict["large"] = False
            cluster_dict["x_coef"] = np.ones((5, 1), dtype=np.float32)
            cluster_dict["y_coef"] = np.ones((5, 1), dtype=np.float32)
            cluster_dict["x_kernel_index"] = 0
            cluster_dict["y_kernel_index"] = 0

        return cluster_id

    

    def add_exp_to_cluster(self, exp_id, cluster_id):
        cluster = self.clusters[cluster_id]
        cluster["members"].add(exp_id)
        self.exps[exp_id]["cluster"] = cluster_id
        x_loglikelihoods = self.exps[exp_id]["x_loglikelihoods"]
        y_loglikelihoods = self.exps[exp_id]["y_loglikelihoods"]
        cluster["x_loglikelihoods"] += x_loglikelihoods
        cluster["y_loglikelihoods"] += y_loglikelihoods
        new_x_loglikelihoods = cluster["x_loglikelihoods"]
        new_y_loglikelihoods = cluster["y_loglikelihoods"]
        x_weights = np.exp(new_x_loglikelihoods - np.max(new_x_loglikelihoods))
        y_weights = np.exp(new_y_loglikelihoods - np.max(new_y_loglikelihoods))
        cluster["x_weights"] = x_weights / np.sum(x_weights)
        cluster["y_weights"] = y_weights / np.sum(y_weights)
        cluster["size"] += 1
        cluster["data_size"] += self.exps[exp_id]["data_size"]

        if cluster["data_size"] >= self.threshold:
            cluster["large"] = False
            cluster["x_kernel_index"] = int(np.argmax(x_weights))
            cluster["y_kernel_index"] = int(np.argmax(y_weights))
            x_coef, y_coef = self.fit_coef(cluster_id)
            cluster["x_coef"] = x_coef
            cluster["y_coef"] = y_coef



    def remove_exp_from_cluster(self, exp_id, cluster_id):
        cluster = self.clusters[cluster_id]
        if cluster["size"] == 1:
            self.empty_cluster_ids.append(cluster_id)
            del self.clusters[cluster_id]
            return

        cluster["members"].discard(exp_id)
        x_loglikelihoods = self.exps[exp_id]["x_loglikelihoods"]
        y_loglikelihoods = self.exps[exp_id]["y_loglikelihoods"]
        cluster["x_loglikelihoods"] -= x_loglikelihoods
        cluster["y_loglikelihoods"] -= y_loglikelihoods
        new_x_loglikelihoods = cluster["x_loglikelihoods"]
        new_y_loglikelihoods = cluster["y_loglikelihoods"]
        x_weights = np.exp(new_x_loglikelihoods - np.max(new_x_loglikelihoods))
        y_weights = np.exp(new_y_loglikelihoods - np.max(new_y_loglikelihoods))
        cluster["x_weights"] = x_weights / np.sum(x_weights)
        cluster["y_weights"] = y_weights / np.sum(y_weights)
        cluster["size"] -= 1
        cluster["data_size"] -= self.exps[exp_id]["data_size"]

        if cluster["data_size"] >= self.threshold:
            cluster["x_kernel_index"] = int(np.argmax(x_weights))
            cluster["y_kernel_index"] = int(np.argmax(y_weights))
            x_coef, y_coef = self.fit_coef(cluster_id)
            cluster["x_coef"] = x_coef
            cluster["y_coef"] = y_coef
        else:
            cluster["large"] = False


    
    def fit_coef(self, cluster_id):
        cluster = self.clusters[cluster_id]
        x_noisy, _, x_length_scale, x_prior_var, x_noise_ratio = self.x_priors[cluster["x_kernel_index"]]
        y_noisy, _, y_length_scale, y_prior_var, y_noise_ratio = self.y_priors[cluster["y_kernel_index"]]
        x_coors = [self.exps[exp_id]["x_data"][:, :-1] for exp_id in cluster["members"]]
        x_targets = [self.exps[exp_id]["x_data"][:, -1].reshape(-1, 1) for exp_id in cluster["members"]]
        y_coors = [self.exps[exp_id]["y_data"][:, :-1] for exp_id in cluster["members"]]
        y_targets = [self.exps[exp_id]["y_data"][:, -1].reshape(-1, 1) for exp_id in cluster["members"]]

        if x_noisy:
            A = sum([x_coor.T @ x_coor for x_coor in x_coors])
            B = sum([x_coor.T @ x_target for x_coor, x_target in zip(x_coors, x_targets)])
            x_coef = lg.inv(A) @ B
        else:
            inverse_covs = [lg.inv(
                (self.kernel(x_coor, length_scales=x_length_scale) + np.eye(len(x_coor))*x_noise_ratio)*x_prior_var
                ) for x_coor in x_coors]
            A = sum([x_coor.T @ inverse_cov @ x_coor for x_coor, inverse_cov in zip(x_coors, inverse_covs)])
            B = sum([x_coor.T @ inverse_cov @ x_target for x_coor, x_target, inverse_cov in zip(x_coors, x_targets, inverse_covs)])
            x_coef = lg.inv(A) @ B

        if y_noisy:
            A = sum([y_coor.T @ y_coor for y_coor in y_coors])
            B = sum([y_coor.T @ y_target for y_coor, y_target in zip(y_coors, y_targets)])
            y_coef = lg.inv(A) @ B
        else:
            inverse_covs = [lg.inv(
                (self.kernel(y_coor, length_scales=y_length_scale) + np.eye(len(y_coor))*y_noise_ratio)*y_prior_var
                ) for y_coor in y_coors]
            A = sum([y_coor.T @ inverse_cov @ y_coor for y_coor, inverse_cov in zip(y_coors, inverse_covs)])
            B = sum([y_coor.T @ inverse_cov @ y_target for y_coor, y_target, inverse_cov in zip(y_coors, y_targets, inverse_covs)])
            y_coef = lg.inv(A) @ B

        return x_coef, y_coef



    def gibbs_sweep(self, return_log_prob=False):
        for exp_id in self.exps:
            exp = self.exps[exp_id]
            log_probs = np.zeros(len(self.clusters)+1, dtype=np.float32)
            clusters = np.zeros(len(self.clusters)+1, dtype=np.int32)
            log_probs[0] = exp["log_self_prob"] + math.log(self.alpha)
            for index, cluster_id in enumerate(self.clusters, 1):
                solo_cluster, log_prob = self.calculate_likelihood(exp_id, cluster_id)
                if solo_cluster:
                    log_probs[index] = log_probs[0]
                    log_probs[0] = -np.inf
                elif exp["cluster"] == cluster_id:
                    log_probs[index] = log_prob + math.log(self.clusters[cluster_id]["size"] - 1)
                else:
                    log_probs[index] = log_prob + math.log(self.clusters[cluster_id]["size"])

                clusters[index] = cluster_id
            
            log_probs -= np.max(log_probs)
            probs = np.exp(log_probs)
            probs /= np.sum(probs)
            selected_cluster = np.random.choice(clusters, p=probs)
            
            original_cluster_id = exp["cluster"]
            if selected_cluster != original_cluster_id:
                self.remove_exp_from_cluster(exp_id, original_cluster_id)
                if selected_cluster:
                    self.add_exp_to_cluster(exp_id, int(selected_cluster))
                else:
                    exp["cluster"] = self.alloc_cluster(exp_id)
        
        if not return_log_prob:
            return 
        
        log_prob = 0
        for cluster in self.clusters.values():
            log_prob += math.lgamma(cluster["size"]) + math.log(self.alpha)
            if cluster["large"]:
                x_noisy, _, x_length_scale, x_prior_var, x_noise_ratio = self.x_priors[cluster["x_kernel_index"]]
                y_noisy, _, y_length_scale, y_prior_var, y_noise_ratio = self.y_priors[cluster["y_kernel_index"]]
                x_coef = cluster["x_coef"]
                y_coef = cluster["y_coef"]
                for exp_id in cluster["members"]:

                    x_coor = self.exps[exp_id]["x_data"][:, :-1]
                    x_target = self.exps[exp_id]["x_data"][:, -1].reshape(-1, 1)
                    y_coor = self.exps[exp_id]["y_data"][:, :-1]
                    y_target = self.exps[exp_id]["y_data"][:, -1].reshape(-1, 1)

                    x_diff = x_target - x_coor @ x_coef
                    y_diff = y_target - y_coor @ y_coef
                    data_size = self.exps[exp_id]["data_size"]

                    if x_noisy:
                        x_loglikelihood = -0.5*(x_diff / (x_prior_var*x_noise_ratio + x_prior_var)).T @ x_diff - \
                            0.5*data_size*math.log(x_prior_var*x_noise_ratio + x_prior_var)
                    else:
                        cov = x_prior_var*self.kernel(x_coor, length_scales=x_length_scale) + np.eye(data_size)*x_noise_ratio*x_prior_var
                        x_loglikelihood = -0.5*(x_diff.T @ lg.inv(cov) @ x_diff)
                        sign, log_det = lg.slogdet(cov)
                        log_det = log_det if sign else -100
                        x_loglikelihood -= 0.5*log_det

                    if y_noisy:
                        y_loglikelihood = -0.5*(y_diff / (y_prior_var*y_noise_ratio + y_prior_var)).T @ y_diff - \
                            0.5*data_size*math.log(y_prior_var*y_noise_ratio + y_prior_var)
                    else:
                        cov = y_prior_var*self.kernel(y_coor, length_scales=y_length_scale) + np.eye(data_size)*y_noise_ratio*y_prior_var
                        y_loglikelihood = -0.5*(y_diff.T @ lg.inv(cov) @ y_diff)
                        sign, log_det = lg.slogdet(cov)
                        log_det = log_det if sign else -100
                        y_loglikelihood -= 0.5*log_det

                    log_prob += float(x_loglikelihood + y_loglikelihood)
            else:
                x_log_likelihoods = cluster["x_loglikelihoods"]
                x_weights = cluster["x_weights"]
                log_prob += np.max(x_log_likelihoods)
                log_prob += np.log(np.sum(x_weights * np.exp(x_log_likelihoods - np.max(x_log_likelihoods))))

                y_log_likelihoods = cluster["y_loglikelihoods"]
                y_weights = cluster["y_weights"]
                log_prob += np.max(y_log_likelihoods)
                log_prob += np.log(np.sum(y_weights * np.exp(y_log_likelihoods - np.max(y_log_likelihoods))))

        return np.float32(log_prob)
        


    def calculate_likelihood(self, exp_id, cluster_id):
        exp = self.exps[exp_id]
        cluster = self.clusters[cluster_id]
        solo_cluster = False

        if exp["cluster"] == cluster_id and cluster["size"] == 1:
            solo_cluster = True
            log_prob = 0
        elif exp["cluster"] == cluster_id and (cluster["data_size"] - exp["data_size"]) < self.threshold:
            x_loglikelihoods = cluster["x_loglikelihoods"] - exp["x_loglikelihoods"]
            y_loglikelihoods = cluster["y_loglikelihoods"] - exp["y_loglikelihoods"]

            x_weights = np.exp(x_loglikelihoods - np.max(x_loglikelihoods))
            y_weights = np.exp(y_loglikelihoods - np.max(y_loglikelihoods))
            x_weights /= np.sum(x_weights)
            y_weights /= np.sum(y_weights)

            x_log_scale = np.max(exp["x_loglikelihoods"])
            y_log_scale = np.max(exp["y_loglikelihoods"])
            x_prob = np.sum(x_weights * np.exp(exp["x_loglikelihoods"] - x_log_scale))
            y_prob = np.sum(y_weights * np.exp(exp["y_loglikelihoods"] - y_log_scale))
            prob = x_prob * y_prob
            log_prob = x_log_scale + y_log_scale + (math.log(prob) if prob else -40)

        elif not cluster["large"]:
            x_log_scale = np.max(exp["x_loglikelihoods"])
            y_log_scale = np.max(exp["y_loglikelihoods"])
            x_prob = np.sum(cluster["x_weights"] * np.exp(exp["x_loglikelihoods"] - x_log_scale))
            y_prob = np.sum(cluster["y_weights"] * np.exp(exp["y_loglikelihoods"] - y_log_scale))
            prob = x_prob * y_prob
            log_prob = x_log_scale + y_log_scale + (math.log(prob) if prob else -40)
        else:
            x_noisy, _, x_length_scale, x_prior_var, x_noise_ratio = self.x_priors[cluster["x_kernel_index"]]
            y_noisy, _, y_length_scale, y_prior_var, y_noise_ratio = self.y_priors[cluster["y_kernel_index"]]

            x_coor = self.exps[exp_id]["x_data"][:, :-1]
            x_target = self.exps[exp_id]["x_data"][:, -1].reshape(-1, 1)
            y_coor = self.exps[exp_id]["y_data"][:, :-1]
            y_target = self.exps[exp_id]["y_data"][:, -1].reshape(-1, 1)

            x_diff = x_target - x_coor @ cluster["x_coef"]
            y_diff = y_target - y_coor @ cluster["y_coef"]
            data_size = exp["data_size"]

            if x_noisy:
                x_loglikelihood = -0.5*(x_diff / (x_prior_var*x_noise_ratio + x_prior_var)).T @ x_diff - \
                    0.5*data_size*math.log(x_prior_var*x_noise_ratio + x_prior_var)
            else:
                cov = x_prior_var*self.kernel(x_coor, length_scales=x_length_scale) + np.eye(data_size)*x_noise_ratio*x_prior_var
                x_loglikelihood = -0.5*(x_diff.T @ lg.inv(cov) @ x_diff)
                sign, log_det = lg.slogdet(cov)
                log_det = log_det if sign else -100
                x_loglikelihood -= 0.5*log_det

            if y_noisy:
                y_loglikelihood = -0.5*(y_diff / (y_prior_var*y_noise_ratio + y_prior_var)).T @ y_diff - \
                    0.5*data_size*math.log(y_prior_var*y_noise_ratio + y_prior_var)
            else:
                cov = y_prior_var*self.kernel(y_coor, length_scales=y_length_scale) + np.eye(data_size)*y_noise_ratio*y_prior_var
                y_loglikelihood = -0.5*(y_diff.T @ lg.inv(cov) @ y_diff)
                sign, log_det = lg.slogdet(cov)
                log_det = log_det if sign else -100
                y_loglikelihood -= 0.5*log_det

            log_prob = x_loglikelihood + y_loglikelihood

        return solo_cluster, log_prob


    def freeze(self):
        self.backup_clusters = {int(key): {i: copy.deepcopy(j) for i, j in value.items()} for key, value in self.clusters.items()}
        self.backup_next_cluster_id = np.int32(self.next_cluster_id)
        self.backup_empty_cluster_ids = self.empty_cluster_ids.copy()
    


    def MAP(self, rounds=20):
        for r in range(rounds):
            new_log_prob = self.gibbs_sweep(return_log_prob=True)
            if new_log_prob > self.log_prob:
                self.freeze()
                self.log_prob = new_log_prob
        
        self.clusters = {int(key): {i: copy.deepcopy(j) for i, j in value.items()} for key, value in self.backup_clusters.items()}
        self.next_cluster_id = int(self.backup_next_cluster_id)
        self.empty_cluster_ids = [i for i in self.backup_empty_cluster_ids]
        for cluster_id, cluster in self.clusters.items():
            for exp_id in cluster["members"]:
                self.exps[exp_id]["cluster"] = cluster_id



    def save(self, dir_path=None):
        dir_path = "archive_xy" if dir_path is None else dir_path
        base_configs = {"alpha": self.alpha, 
                        "next_exp_id": self.next_exp_id, 
                        "next_cluster_id": self.next_cluster_id, 
                        "empty_cluster_ids": self.empty_cluster_ids.copy(), 
                        "empty_exp_ids": self.empty_exp_ids.copy(), 
                        "threshold": self.threshold} # <--
        
        exp_seps = [] 
        exp_length = 0
        exps_x_data = []
        exps_y_data = []

        for exp_id, exp in self.exps.items():
            exp_seps.append((exp_id, exp_length, exp_length + exp["data_size"]))
            exp_length += exp["data_size"]
            exps_x_data.append(exp["x_data"])
            exps_y_data.append(exp["y_data"])
        base_configs["exp_seps"] = exp_seps
        
        exps_x_data = np.concatenate(exps_x_data, axis=0)[None, :, :]
        exps_y_data = np.concatenate(exps_y_data, axis=0)[None, :, :]
        exp_data = np.concatenate((exps_x_data, exps_y_data), axis=0) # <--

        clusters_data = {} # <--
        for cluster_id, cluster in self.clusters.items():
            cluster_info = {}
            for key, value in cluster.items():
                if type(value) == np.ndarray:
                    cluster_info[key] = value.tolist()
                elif type(value) == set:
                    cluster_info[key] = list(value)
                else:
                    cluster_info[key] = value
            
            clusters_data[cluster_id] = cluster_info
        
        if not os.path.exists(dir_path):
            os.mkdir(dir_path)

        np.save(os.path.join(dir_path, "exp_data.npy"), exp_data)
        np.save(os.path.join(dir_path, "GP_x.npy"), self.x_prior_data)
        np.save(os.path.join(dir_path, "GP_y.npy"), self.y_prior_data)

        with open(os.path.join(dir_path, "base_configs.json"), "w") as file:
            json.dump(base_configs, file)

        with open(os.path.join(dir_path, "cluster_data.json"), "w") as file:
            json.dump(clusters_data, file)
        


    def load(self, dir_path):
        prior_path = {"x": os.path.join(dir_path, "GP_x.npy"), 
                      "y": os.path.join(dir_path, "GP_y.npy")}
        self.load_priors(prior_path)

        with open(os.path.join(dir_path, "base_configs.json"), "r") as file:
            base_configs = json.load(file)

        with open(os.path.join(dir_path, "cluster_data.json"), "r") as file:
            cluster_data = json.load(file)
        
        exp_data = np.load(os.path.join(dir_path, "exp_data.npy"))

        self.alpha = base_configs["alpha"]
        self.exps = {}
        self.clusters = {}
        self.next_exp_id = base_configs["next_exp_id"]
        self.next_cluster_id = base_configs["next_cluster_id"]
        self.empty_cluster_ids = base_configs["empty_cluster_ids"]
        self.empty_exp_ids = base_configs["empty_exp_ids"]
        self.threshold = base_configs["threshold"]

        exp_seps = base_configs["exp_seps"]
        exps = []
        ids = []

        for exp_id, start, end in exp_seps:
            x_exp, y_exp = exp_data[:, start: end, :].copy()
            exps.append({"x": x_exp, "y": y_exp})
            ids.append(exp_id)

        self.read_experiences(exps, ids)

        for cluster_id, cluster in cluster_data.items():
            updated_dict = {}
            for key, value in cluster.items():
                if key == "members":
                    updated_dict[key] = set(value)
                    for exp_id in value:
                        self.exps[exp_id]["cluster"] = cluster_id
                elif type(value) == list:
                    updated_dict[key] = np.array(value)
            
            cluster.update(updated_dict)
            self.clusters[cluster_id] = cluster
    

    def build_adaptor(self):
        total_size = 0
        x_gps = []
        y_gps = []
        prior_weights = []
        for cluster in self.clusters.values():
            if cluster["data_size"] >= 15:
                total_size += cluster["size"]

        if not total_size:
            raise Exception("Not enough data to build adaptor")
        
        for cluster in self.clusters.values():
            if cluster["data_size"] < 15:
                continue

            prior_weights.append(cluster["size"]/total_size)
            x_gp = [copy.deepcopy(i) for i in self.x_priors[cluster["x_kernel_index"]]]
            y_gp = [copy.deepcopy(i) for i in self.y_priors[cluster["y_kernel_index"]]]

            if cluster["large"]:
                x_gp[1] = cluster["x_coef"].copy()
                y_gp[1] = cluster["y_coef"].copy()

            x_gps.append(x_gp)
            y_gps.append(y_gp)
        
        adaptor = Adaptor_xy((x_gps, y_gps), np.array(prior_weights))
        return adaptor
                






In [26]:
coor_x = np.load("coor_x.npy")
coor_y = np.load("coor_y.npy")
baseline = np.load("filtered_baseline.npy")

# print(coor_x.shape)

exps = []
for file in os.listdir(r"../exps/dmg1"):
    data = np.load(os.path.join(r"../exps/dmg1", file))
    exp = {}
    indexes = np.int32(data[:, 0])
    x_targets = (data[:, 1] - baseline[indexes, 0]).reshape(-1, 1)
    # print(baseline[indexes, 0])
    x_coors = coor_x[indexes]
    # print(x_targets.shape, x_coors.shape)
    exp["x"] = np.hstack((x_coors, x_targets))
    y_targets = (data[:, 2] - baseline[indexes, 1]).reshape(-1, 1)
    y_coors = coor_y[indexes]
    exp["y"] = np.hstack((y_coors, y_targets))

    exps.append(exp)


for file in os.listdir(r"../exps/dmg0"):
    data = np.load(os.path.join(r"../exps/dmg0", file))
    exp = {}
    indexes = np.int32(data[:, 0])
    x_targets = (data[:, 1] - baseline[indexes, 0]).reshape(-1, 1)
    x_coors = coor_x[indexes]
    # print(x_targets.shape, x_coors.shape)
    exp["x"] = np.hstack((x_coors, x_targets))
    y_targets = (data[:, 2] - baseline[indexes, 1]).reshape(-1, 1)
    y_coors = coor_y[indexes]
    exp["y"] = np.hstack((y_coors, y_targets))

    exps.append(exp)

for file in os.listdir(r"../exps/dmg2"):
    data = np.load(os.path.join(r"../exps/dmg2", file))
    exp = {}
    indexes = np.int32(data[:, 0])
    x_targets = (data[:, 1] - baseline[indexes, 0]).reshape(-1, 1)
    x_coors = coor_x[indexes]
    # print(x_targets.shape, x_coors.shape)
    exp["x"] = np.hstack((x_coors, x_targets))
    y_targets = (data[:, 2] - baseline[indexes, 1]).reshape(-1, 1)
    y_coors = coor_y[indexes]
    exp["y"] = np.hstack((y_coors, y_targets))

    exps.append(exp)


for file in os.listdir(r"../exps/dmg3"):
    data = np.load(os.path.join(r"../exps/dmg3", file))
    exp = {}
    indexes = np.int32(data[:, 0])
    x_targets = (data[:, 1] - baseline[indexes, 0]).reshape(-1, 1)
    x_coors = coor_x[indexes]
    # print(x_targets.shape, x_coors.shape)
    exp["x"] = np.hstack((x_coors, x_targets))
    y_targets = (data[:, 2] - baseline[indexes, 1]).reshape(-1, 1)
    y_coors = coor_y[indexes]
    exp["y"] = np.hstack((y_coors, y_targets))

    exps.append(exp)


In [27]:
archive = Archive_xy()

prior_path = {"x": "GP_x.npy", "y": "GP_y.npy"}

archive.load_priors(prior_path)

archive.read_experiences(exps)

In [28]:

print(archive.next_exp_id)


51


In [29]:
archive.gibbs_sweep()


In [30]:
print(len(archive.clusters))

20


In [31]:
for i in range(20):
    archive.gibbs_sweep()

print(len(archive.clusters))

8


In [32]:
for cluster in archive.clusters.values():
    print(cluster["size"], cluster["data_size"])

10 147
10 131
6 78
4 49
3 51
10 126
5 57
2 20


In [33]:
def find_dmg(exp_id):
    if exp_id < 11:
        return 1
    if exp_id < 31:
        return 0
    if exp_id < 36:
        return 2
    return 3

In [34]:
archive.MAP(rounds=200)

In [35]:
print(archive.log_prob)

651.85724


In [36]:
cluster_data = []
for cluster in archive.clusters.values():
    sub_data = np.zeros(4)
    dmgs = [find_dmg(exp_id) for exp_id in cluster["members"]]
    print(dmgs.count(0) / len(dmgs), dmgs.count(1) / len(dmgs), dmgs.count(2) / len(dmgs), dmgs.count(3) / len(dmgs), "  ", cluster["size"])
    sub_data[0] = dmgs.count(0)
    sub_data[1] = dmgs.count(1)
    sub_data[2] = dmgs.count(2)
    sub_data[3] = dmgs.count(3)
    cluster_data.append(sub_data.copy())

    
    if cluster["size"] == 1:
        for exp_id in cluster["members"]:
            if find_dmg(exp_id) == 2:
                print(archive.exps[exp_id]["data_size"])

cluster_data = np.vstack(cluster_data)
print(cluster_data.shape)
np.save("cluster_data_xy.npy", cluster_data)

0.0 1.0 0.0 0.0    7
1.0 0.0 0.0 0.0    11
1.0 0.0 0.0 0.0    5
0.0 0.0 0.0 1.0    11
0.0 0.0 0.0 1.0    4
1.0 0.0 0.0 0.0    2
0.0 0.0 1.0 0.0    3
0.0 1.0 0.0 0.0    3
1.0 0.0 0.0 0.0    1
1.0 0.0 0.0 0.0    1
0.0 0.0 1.0 0.0    2
(11, 4)


In [37]:
# archive.save()
# del archive

In [38]:
for cluster in archive.clusters.values():
    print(cluster["size"], cluster["data_size"])

7 96
11 137
5 72
11 138
4 45
2 29
3 51
3 51
1 10
1 10
2 20


In [39]:
# new_archive = Archive_xy()
# # new_archive.load_priors(prior_path)
# new_archive.load("archive")
# for cluster in new_archive.clusters.values():
#     print(cluster["size"], cluster["data_size"])